In [ ]:
!pip install plotly==5.6.0
!pip install -U kaleido

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
results = pd.read_csv('data/results_cleaned.csv')

In [ ]:
metrics = results[['fault_profile', 'availability', 'mut', 'mdt', 'mtbf', 'finished_jobs', 'submitted_jobs', 'enhancement']]
metrics['finished_jobs_ratio'] = (
    metrics['finished_jobs'] / metrics['submitted_jobs'])
metrics = metrics.drop(columns=['finished_jobs', 'submitted_jobs'])
metrics.head()

In [ ]:
# rename to relative metrics
metrics = metrics.rename(columns={'mut': 'rel_MUT', 'mdt': 'rel_MDT', 'mtbf': 'rel_MTBF'})
metrics

In [ ]:
# Baseline metrics
baseline_metrics = metrics[metrics['enhancement'] == 'None']
baseline_metrics

In [ ]:
# create long data from wide
metrics_long = metrics.melt(id_vars=['fault_profile', 'enhancement'], var_name='metric')
metrics_long.head()
metrics_long

In [ ]:
# Baseline metrics long
baseline_metrics_long = metrics_long[metrics_long['enhancement'] == 'None']
baseline_metrics_long

In [ ]:
# ALL BASELINE RESULTS IN ONE Radar
fig = px.line_polar(baseline_metrics_long, r="value", theta="metric", color="fault_profile", line_close=True, color_discrete_sequence=px.colors.qualitative.Dark24)
# fig.update_traces(fill='toself')
fig.show()

In [ ]:
# BASELINE METRICS - 1 Radar PLOT / PROFILE (no subplots)

# fault_profiles = baseline_metrics['fault_profile'].tolist()

# for profile in fault_profiles:
#     data = baseline_metrics_long[baseline_metrics_long['fault_profile'] == profile]
    
#     fig = px.line_polar(
#         data,
#         r="value",
#         theta='metric',
#         line_close=True,
#         title=f'Fault Profile: {profile}'
#     )
#     fig.update_traces(fill='toself')
#     fig.show()

In [ ]:
# BASELINE METRICS - GROUPED BARCHART
import plotly.graph_objects as go

fault_profiles = baseline_metrics['fault_profile'].tolist()
bars = []
for profile in fault_profiles:
    data = baseline_metrics_long[baseline_metrics_long['fault_profile'] == profile].sort_values(by='metric')
#     print(data)
    bars.append(go.Bar(name=profile, x=data['metric'], y=data['value']))

fig = go.Figure(data=bars)
fig.update_layout(barmode='group')

fig.show()

fig.write_image('images/baseline_grouped_barchart.png', engine='kaleido',scale=2)

# Enhancements compared to baseline results

## Kafka

In [ ]:
### only use_kafka and baseline metrics (on same fault profiles)
kafka_only_metrics_long = metrics_long[metrics_long['enhancement'] == 'Kafka']
kafka_only_metrics_long

In [ ]:
kafka_only_profiles = kafka_only_metrics_long['fault_profile'].unique().tolist()
kafka_only_profiles

In [ ]:
kafka_or_baseline_filter = metrics_long['enhancement'].isin(['None', 'Kafka'])
kafka_with_base_metrics_long = metrics_long[ \
    (kafka_or_baseline_filter & (metrics_long['fault_profile'].isin(kafka_only_profiles)))]
kafka_with_base_metrics_long

In [ ]:
for profile in kafka_only_profiles:
    data = kafka_with_base_metrics_long[kafka_with_base_metrics_long['fault_profile'] == profile]
    
    fig = px.line_polar(
        data,
        r="value",
        theta='metric',
        color="enhancement",
        line_close=True,
        title=f'Fault Profile: {profile}'
    )
    fig.update_traces(fill='toself')
    fig.show()
    
    fig.write_image(f'images/kafka_with_base_{profile}.png', engine='kaleido',scale=2)

## Heartbeats

In [ ]:
heartbeats_only_metrics_long = metrics_long[metrics_long['enhancement'] == 'Heartbeats']
heartbeats_only_metrics_long

In [ ]:
heartbeats_only_profiles = heartbeats_only_metrics_long['fault_profile'].unique().tolist()
heartbeats_only_profiles

In [ ]:
heartbeats_or_baseline_filter = metrics_long['enhancement'].isin(['None', 'Heartbeats'])
heartbeats_with_base_metrics_long = metrics_long[ \
    (heartbeats_or_baseline_filter & (metrics_long['fault_profile'].isin(heartbeats_only_profiles)))]
heartbeats_with_base_metrics_long

In [ ]:
for profile in heartbeats_only_profiles:
    data = heartbeats_with_base_metrics_long[heartbeats_with_base_metrics_long['fault_profile'] == profile]
    
    fig = px.line_polar(
        data,
        r="value",
        theta='metric',
        color="enhancement",
        line_close=True,
        title=f'Fault Profile: {profile}'
    )
    fig.update_traces(fill='toself')
    fig.show()
    fig.write_image(f'images/heartbeats_with_base_{profile}.png', engine='kaleido',scale=2)

## Kafka and Heartbeats

In [ ]:
kafka_and_hb_metrics_long = metrics_long[metrics_long['enhancement'] == 'Kafka_And_Heartbeats']
kafka_and_hb_metrics_long

In [ ]:
kafka_and_hb_profiles = kafka_and_hb_metrics_long['fault_profile'].unique().tolist()
kafka_and_hb_profiles

In [ ]:
kafka_and_hb_or_baseline_filter = metrics_long['enhancement'].isin(['None', 'Kafka_And_Heartbeats'])
kafka_and_hb_with_base_metrics_long = metrics_long[ \
    (kafka_and_hb_or_baseline_filter & (metrics_long['fault_profile'].isin(kafka_and_hb_profiles)))]
kafka_and_hb_with_base_metrics_long

In [ ]:
for profile in kafka_and_hb_profiles:
    data = kafka_and_hb_with_base_metrics_long[kafka_and_hb_with_base_metrics_long['fault_profile'] == profile]
    
    fig = px.line_polar(
        data,
        r="value",
        theta='metric',
        color="enhancement",
        line_close=True,
        title=f'Fault Profile: {profile}'
    )
    fig.update_traces(fill='toself')
    fig.show()
    fig.write_image(f'images/kafka_and_hb_with_base_{profile}.png', engine='kaleido',scale=2)